# E2-4 Tic Tac Toe Wiser - Defend

This is a game, in which AI starts playing and tries to protect against the opponent winning the game. At each move, AI checks for cases, in which the opponent can win and tries to stop them.

## Initialization

In [1]:
# Initialize the players and signs
EMPTY = '.'
AI = 'X'
HUMAN = 'O'

In [2]:
# print the board, leave an empty lines for visibility
def print_board(board):
    print(" ")
    print(' '.join(board[:3]))
    print(' '.join(board[3:6]))
    print(' '.join(board[6:]))
    print(" ")


In [3]:
# Initialize the board
board = EMPTY * 9
print_board(board)

 
. . .
. . .
. . .
 


In [4]:
# Define all possible winning combinations
win_cases = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [0, 3, 6],
    [1, 4, 7],
    [2, 5, 8],
    [0, 4, 8],
    [2, 4, 6]
]

##  Supporting Functions

#### Function improved
AI is interested in save options

In [5]:
# Get all available save moves on the board from this position in a list 'move_list'
def all_possible_moves(board, sign):
    # start with empty list
    move_list = []
    # travers the board, i-index, v-sign
    for i, v in enumerate(board):
        if v == EMPTY:
            new_board = board[:i] + sign + board[i+1:]
            move_list.append(new_board)
            if game_won_by(new_board) == AI:
                return [new_board]
    # add new check
    if sign == AI:
        safe_moves = []
        # it is not safe to leave empty a cell, where the opponent can win
        for move in move_list:
            if not can_win(move, HUMAN):
                safe_moves.append(move)
        return safe_moves if len(safe_moves) > 0 else move_list[0:1]
    else:
        return move_list

In [6]:
# A winning game is if any of win-cases occurs
def game_won_by(board):
    for i in win_cases:
        if board[i[0]] == board[i[1]] == board[i[2]] != EMPTY:
            # win-case, return the sign, which wins
            return board[i[0]]
    return EMPTY

#### New function

In [7]:
# Before make a move, iterate through the options and check where the player just would win
def can_win(board, sign):
    next_moves = all_possible_moves(board, sign)
    for next_move in next_moves:
        if game_won_by(next_move) == sign:
            return True
    return False

## Play The Game

###  Game Moves

#### Human move

In [8]:
# Human move approach still the same
def human_move(board, row, column):
    # get the index of the cell the user selected: 2D -> 1D 
    index = 3 * (row - 1) + (column - 1)
    #  if this cell is empty, make the user move, otherwise do nothing
    if board[index] == EMPTY:
        # place HUMAN sign on board[index]
        return board[:index] + HUMAN + board[index+1:]
    return board

#### AI move:  ai_move() is improved
Save preffered moves 

In [9]:
# AI makes a move from all available SAFE moves
from random import choice
def ai_move(board):
    options = all_possible_moves(board, AI)
    for option in options:
        if game_won_by(option) == AI:
            return option
    safe_moves = []
    for option in options:
        if not can_win(option, HUMAN):
            safe_moves.append(option)
    if len(safe_moves) > 0: 
        return choice(safe_moves) 
    return options[0]  

In [10]:
# Play the game
def game():
    # start from empty board
    board = EMPTY * 9
    empty_cell_count = 9
    end_flag = False
    
    while empty_cell_count > 0 and not end_flag:        
        # Player AI (always odd number of options)
        if empty_cell_count % 2 == 1:
            board = ai_move(board)
        else:
            # Human player
            row = int(input('Enter row: '))
            col = int(input('Enter column: '))
            board = human_move(board, row, col)
            
        # Print current board status    
        print_board(board)
        
        # Check if someone wins already, update the flag
        end_flag = game_won_by(board) != EMPTY
        
        # Count how many empty cells left
        empty_cell_count = board.count(EMPTY)      
        # empty_cell_count = sum(1 for cell in board if cell == EMPTY_SIGN)
     
    print('Game ended. Winner: ', game_won_by(board))

In [11]:
# Run the game
game()

 
. . .
. . X
. . .
 
Enter row: 2
Enter column: 2
 
. . .
. O X
. . .
 
 
. . X
. O X
. . .
 
Enter row: 3
Enter column: 3
 
. . X
. O X
. . O
 
 
X . X
. O X
. . O
 
Enter row: 1
Enter column: 2
 
X O X
. O X
. . O
 
 
X O X
. O X
. X O
 
Enter row: 2
Enter column: 1
 
X O X
O O X
. X O
 
 
X O X
O O X
X X O
 
Game ended. Winner:  .


## Game Analysis

We want to know how many are the possible combinations of moves and how many times each player can win the game.

In [12]:
# all moves for all possible states for this player
def all_moves(board_list, sign):
    move_list = []
    for board in board_list:
        move_list.extend(all_possible_moves(board, sign))
    return move_list

In [13]:
# All wins of each player separated in two new lists, draws remains in the old list
def player_wins(move_list, ai_wins, human_wins):
    for board in move_list:
        won_by = game_won_by(board)
        if won_by == AI:
            ai_wins.append(board)
            move_list.remove(board)
        elif won_by == HUMAN:
            human_wins.append(board)
            move_list.remove(board)

In [14]:
# At each step count the available moves
def count_possibilities():
    board = EMPTY * 9
    move_list = [board]
    ai_wins = []
    human_wins = []
    for i in range(9):
        print('Step ' + str(i) + ': possible moves = ' + str(len(move_list)))
        sign = AI if i % 2 == 0 else HUMAN
        move_list = all_moves(move_list, sign)
        player_wins(move_list, ai_wins, human_wins)
    print('First player wins: ' + str(len(ai_wins)))
    print('Second player wins: ' + str(len(human_wins)))
    print('Draw', str(len(move_list)))
    print('Total', str(len(ai_wins) + len(human_wins) + len(move_list)))

In [15]:
count_possibilities()

Step 0: possible moves = 1
Step 1: possible moves = 9
Step 2: possible moves = 72
Step 3: possible moves = 504
Step 4: possible moves = 3024
Step 5: possible moves = 5197
Step 6: possible moves = 18606
Step 7: possible moves = 19592
Step 8: possible moves = 30936
First player wins: 20843
Second player wins: 962
Draw 20243
Total 42048
